In [1]:
# 최석재 lingua@naver.com
# 구글 드라이브와 연결
# from google.colab import auth
# auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# 경로 설정
path = '/content/gdrive/MyDrive/pytest/data/eng-kor/'
!ls '/content/gdrive/MyDrive/pytest/data/eng-kor/'

eng-kor_small.txt  eng-kor.txt


In [3]:
# 데이터 확인
# 빠른 진행을 위해 small 데이터로 수행한다.
import pandas as pd
data = pd.read_csv(path+'eng-kor_small.txt', names=['source', 'target'], sep='\t', encoding='utf-8')
print('data length: ', len(data))
print('data sample: ', data.sample(5))

data length:  1000
data sample:                source     target
28           I know.        알아.
499   Grab the rope.  로프를 잡으세요.
447    Please smile.       웃어줘.
615  Watch yourself.       조심해.
109        What for?      뭐 하러?


In [4]:
# 시작부호와 종료부호 부착
# 데이터가 모두 3종이 필요하다. source 언어에는 encoder_input 1개, target 언어에는 decoder_input, decoder_target 2개이다
# encoder는 source 언어를 그대로 사용하면 되나, decoder는 seq2seq의 사용을 위해 <sos>, <eos>를 부착해야 한다
# decoder_input 데이터의 시작에는 <sos>, 문장의 끝에는 <eos>를 부착한다
# decoder_target 데이터는 <eos>만 필요하다
# 어절분리가 되도록 <sos> 뒤에 공백, <eos> 앞에 공백을 두어야 한다
data.target_input = data.target.apply(lambda x : 'sos '+x+' eos')
data.target_target = data.target.apply(lambda x : x+' eos')
print('\ndata.target_input\n', data.target_input)
print('\ndata.target_target\n', data.target_target)


data.target_input
 0                 sos 가. eos
1                sos 안녕. eos
2                sos 뛰어! eos
3                sos 뛰어. eos
4                sos 누구? eos
               ...          
995     sos 노래하는 거 좋아해요? eos
996      sos 노래하는 거 좋아해? eos
997    sos 고양이를 좋아하지 않아? eos
998      sos 꿈은 이루어질 거야. eos
999     sos 모두 그녀를 사랑한다. eos
Name: target, Length: 1000, dtype: object

data.target_target
 0                 가. eos
1                안녕. eos
2                뛰어! eos
3                뛰어. eos
4                누구? eos
             ...        
995     노래하는 거 좋아해요? eos
996      노래하는 거 좋아해? eos
997    고양이를 좋아하지 않아? eos
998      꿈은 이루어질 거야. eos
999     모두 그녀를 사랑한다. eos
Name: target, Length: 1000, dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [5]:
# 문장의 길이 maxlen 설정하기
# source와 target 문장의 최대 길이를 구한다
max_src_len = data.source.apply(lambda x: len(x.split())).max()           # 공백 단위(어절)로 분리하고, 그 어절의 최대 개수를 센다
print('source sentence max length: ', max_src_len)                        # source 문장의 최대 음절 길이로 maxlen을 설정한다
max_tar_len = data.target_input.apply(lambda x: len(x.split())).max()
print('target sentence max length: ', max_tar_len)                        # target 문장의 최대 음절 길이로 maxlen을 설정한다

source sentence max length:  5
target sentence max length:  8


In [6]:
# Data Tokenizing
# 각 단어 종류에 대하여 숫자값을 배당한다
from keras.preprocessing.text import Tokenizer

tokenizer_source = Tokenizer(num_words=None, char_level=False, lower=True)      # Tokenizer 객체 생성. char_level은 False, lower는 True로 한다
tokenizer_source.fit_on_texts(data.source)     	                                # 인덱스를 구축한다
word_index_source = tokenizer_source.word_index                                 # 글자와 인덱스의 쌍을 가져온다

print('\n전체에서 %s개의 고유한 토큰을 찾았습니다.' % len(word_index_source))
print('word_index_source type: ', type(word_index_source))
print('word_index_source: ', word_index_source)


전체에서 786개의 고유한 토큰을 찾았습니다.
word_index_source type:  <class 'dict'>
word_index_source:  {'tom': 1, 'i': 2, 'is': 3, 'you': 4, 'a': 5, "i'm": 6, 'it': 7, 'me': 8, 'they': 9, 'this': 10, 'like': 11, 'the': 12, 'keep': 13, 'we': 14, 'do': 15, 'stop': 16, 'please': 17, "don't": 18, 'be': 19, 'come': 20, 'how': 21, 'was': 22, 'my': 23, 'everyone': 24, 'to': 25, 'up': 26, "it's": 27, 'that': 28, 'he': 29, 'she': 30, 'too': 31, 'here': 32, 'lie': 33, 'are': 34, 'no': 35, 'not': 36, 'can': 37, 'go': 38, 'what': 39, 'sorry': 40, 'everybody': 41, 'have': 42, 'very': 43, 'work': 44, 'died': 45, 'left': 46, 'there': 47, 'nobody': 48, 'old': 49, 'cat': 50, "that's": 51, 'try': 52, 'won': 53, 'get': 54, 'in': 55, 'look': 56, 'smiled': 57, 'him': 58, 'still': 59, 'love': 60, "you're": 61, 'blood': 62, 'red': 63, 'singing': 64, 'at': 65, "isn't": 66, 'who': 67, 'got': 68, 'sit': 69, 'home': 70, 'laughed': 71, 'now': 72, 'take': 73, 'turn': 74, 'want': 75, 'yourself': 76, 'fish': 77, 'something': 78, "c

In [7]:
# target 언어 Tokenizing
# target 언어의 Tokenizer도 target_input으로만 만들면 된다 (target의 output은 같은 언어이므로)
tokenizer_target = Tokenizer(num_words=None, char_level=False, lower=True)      # Tokenizer 객체 생성. 고빈도 어휘만 사용하려면 num_words에 값을 줄 수 있다. char_level은 False로 해야 한다
tokenizer_target.fit_on_texts(data.target_input)     	                          # 인덱스를 구축한다
word_index_target = tokenizer_target.word_index                                 # 단어와 인덱스의 쌍을 가져온다

print('\n전체에서 %s개의 고유한 토큰을 찾았습니다.' % len(word_index_target))
print('word_index_target: ', word_index_target)


전체에서 1206개의 고유한 토큰을 찾았습니다.
word_index_target:  {'sos': 1, 'eos': 2, '톰이': 3, '톰은': 4, '난': 5, '있어': 6, '그': 7, '나는': 8, '계속': 9, '웃었어': 10, '우린': 11, '거짓말': 12, '그만': 13, '내': 14, '와': 15, '좀': 16, '수': 17, '좋아해': 18, '그는': 19, '너무': 20, '하지': 21, '마': 22, '모두': 23, '있어봐': 24, '내가': 25, '해': 26, '이거': 27, '사람들은': 28, '없어': 29, '톰을': 30, '말해': 31, '죽었어': 32, '날': 33, '사람은': 34, '정말': 35, '아무도': 36, '나': 37, '했어': 38, '제발': 39, '봐': 40, '줘': 41, '누군가': 42, '너': 43, '아니야': 44, '우리는': 45, '이렇게': 46, '수가': 47, '이': 48, '이렇게나': 49, '그건': 50, '이겼어': 51, '누가': 52, '떠났어': 53, '잘': 54, '빨리': 55, '그거': 56, '좋아': 57, '이건': 58, '안': 59, '않았어': 60, '모두들': 61, '좋아해요': 62, '그녀는': 63, '걸': 64, '진정해': 65, '들어': 66, '왔어': 67, '조심해': 68, '앉아': 69, '톰한테': 70, '그들이': 71, '거짓말을': 72, '가져': 73, '다시': 74, '한': 75, '조용히': 76, '않아': 77, '넌': 78, '거야': 79, '고양이는': 80, '늙었어': 81, '모두가': 82, '사라졌어': 83, '가': 84, '안녕': 85, '누구': 86, '도와줘': 87, '시작해': 88, '일해': 89, '우리가': 90, '왜': 91, '사람이': 92, '그만해': 93, '톰에게': 94

In [8]:
# Data Sequencing
# 배당된 숫자를 이용하여 각 문장의 문자를 숫자로 치환한다
# source 언어 Sequencing
encoder_input = tokenizer_source.texts_to_sequences(list(data.source))

print('\nResult of encoder_input sequencing: ')
print(data.source[0], encoder_input[0])
print(data.source[1], encoder_input[1])
print(data.source[2], encoder_input[2])


Result of encoder_input sequencing: 
Go. [38]
Hi. [381]
Run! [221]


In [9]:
# target 언어 Sequencing
decoder_input = tokenizer_target.texts_to_sequences(list(data.target_input))
decoder_target = tokenizer_target.texts_to_sequences(list(data.target_target))

print('\nResult of decoder_input sequencing: ')
print(data.target_input[0], decoder_input[0])
print(data.target_input[1], decoder_input[1])
print(data.target_input[2], decoder_input[2])

print('\nResult of decoder_target sequencing: ')
print(data.target_target[0], decoder_target[0])
print(data.target_target[1], decoder_target[1])
print(data.target_target[2], decoder_target[2])


Result of decoder_input sequencing: 
sos 가. eos [1, 84, 2]
sos 안녕. eos [1, 85, 2]
sos 뛰어! eos [1, 176, 2]

Result of decoder_target sequencing: 
가. eos [84, 2]
안녕. eos [85, 2]
뛰어! eos [176, 2]


In [10]:
# Data Padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

print('\nPadding result sample: ')
print(decoder_input.shape) 
print(data.target_input[0], decoder_input[0])


Padding result sample: 
(1000, 8)
sos 가. eos [ 1 84  2  0  0  0  0  0]


In [11]:
# One-Hot-Encoding
# 클래스의 수는 1을 올려주어야 한다(Padding으로 생긴 0을 추가로 받아야 함)
# Word Embedding을 할 대상에는 원-핫 인코딩을 하지 않는다
# 출력층에 대해서만 원-핫 인코딩을 수행한다
from tensorflow.keras.utils import to_categorical
decoder_target = to_categorical(decoder_target, num_classes=len(word_index_target)+1)

print('\nResult of One-Hot Encodded decoder_input sequencing: ')
print(decoder_target.shape)
print(decoder_target[0])


Result of One-Hot Encodded decoder_input sequencing: 
(1000, 8, 1207)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [12]:
# 교사 강요를 이용한 모델 훈련
# Context Vector 만들기
# 훈련용 Encoder, Decoder
from keras.models import Model
from keras.layers import Input, LSTM, Embedding, Dense, Masking

# Encoder - Source
# 입력문의 길이는 문장마다 다르므로 None이다. 출력은 len(word_index_source)+1가 되므로 shape=(None, len(word_index_source)+1)로 할 수 있다
# 하지만, 임베딩 과정에서 다시 한 번 차원이 늘어나게 되므로 Input 단계에서 차원을 하나 줄이기 위하여 출력을 지정하지 않는다 (Embedding 층에서 지정)
encoder_inputs = Input(shape=(None, ), name='encoder_inputs') 
embedded_encoder_inputs = Embedding(input_dim=len(word_index_source)+1, output_dim=256, input_length=None)(encoder_inputs)    # 단어 수준이므로 Embedding 층을 만든다
embedded_encoder_inputs_mask = Masking(mask_value=0.0)(embedded_encoder_inputs) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(units=256, return_state=True)                               # encoder 내부 상태를 decoder로 넘겨주기 위해 return_state=True

encoder_outputs, state_h, state_c = encoder_lstm(embedded_encoder_inputs_mask)  # encoder_outputs는 사용하지 않는다. 임베딩된 인코더 input이 들어간다
encoder_states = [state_h, state_c]                                             # 은닉상태와 셀상태를 받는다

In [13]:
# Decoder - Input
decoder_inputs = Input(shape=(None, ), name='decoder_inputs')
embedded_decoder_inputs = Embedding(input_dim=len(word_index_target)+1, output_dim=256)(decoder_inputs)
embedded_decoder_inputs_mask = Masking(mask_value=0.0)(embedded_decoder_inputs)     # 패딩 0은 연산에서 제외
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)            # decoder의 은닉 상태도 256으로 동일하게 맞춰준다. 매 timestep마다 결과를 return 하기 위해 return_state=True

# Decoder - Ouput
decoder_outputs, _, _ = decoder_lstm(embedded_decoder_inputs_mask, initial_state=encoder_states)  # decoder의 첫 상태를 encoder의 은닉 상태와 셀 상태로 한다
decoder_dense = Dense(len(word_index_target)+1, activation='softmax')               # decoder의 은닉상태와 셀상태는 훈련 과정에서는 사용하지 않는다(_)
decoder_outputs = decoder_dense(decoder_outputs)                                    # 출력층의 크기는 번역문의 글자(또는 단어)가 가질 수 있는 크기이다

In [14]:
# 모델 훈련
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=100, validation_split=0.2)

Epoch 1/100
13/13 [==============================] - 14s 197ms/step - loss: 3.7355 - val_loss: 3.2488
Epoch 2/100
13/13 [==============================] - 0s 14ms/step - loss: 2.1920 - val_loss: 3.4992
Epoch 3/100
13/13 [==============================] - 0s 14ms/step - loss: 2.1026 - val_loss: 3.2468
Epoch 4/100
13/13 [==============================] - 0s 14ms/step - loss: 2.0546 - val_loss: 3.4208
Epoch 5/100
13/13 [==============================] - 0s 14ms/step - loss: 1.9652 - val_loss: 3.3734
Epoch 6/100
13/13 [==============================] - 0s 14ms/step - loss: 1.9042 - val_loss: 3.4073
Epoch 7/100
13/13 [==============================] - 0s 14ms/step - loss: 1.8513 - val_loss: 3.2651
Epoch 8/100
13/13 [==============================] - 0s 14ms/step - loss: 1.7984 - val_loss: 3.3362
Epoch 9/100
13/13 [==============================] - 0s 14ms/step - loss: 1.7776 - val_loss: 3.2386
Epoch 10/100
13/13 [==============================] - 0s 14ms/step - loss: 1.7313 - val_loss: 3.23

In [15]:
# 예측용 Encoder
# 입력된 문장을 인코더에 넣어서 은닉상태와 셀상태를 얻는다
# 인코더가 입력을 받아 자신의 내부 상태를 디코더에 넘겨주는 과정을 하나의 인코더 모델로 만든다
# encoder_inputs, encoder_states는 훈련용에서 구성한 것을 사용한다
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [16]:
# 예측용 Decoder
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태를 이전 상태로 사용
decoder_outputs, state_h, state_c = decoder_lstm(embedded_decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [17]:
# word로부터 idx를 얻는 것을 idx로부터 word를 얻는 것으로 바꿈
index_to_src = dict((i, word) for word, i in word_index_source.items())
index_to_tar = dict((i, word) for word, i in word_index_target.items())
print(index_to_src)
print(index_to_tar)

{1: 'tom', 2: 'i', 3: 'is', 4: 'you', 5: 'a', 6: "i'm", 7: 'it', 8: 'me', 9: 'they', 10: 'this', 11: 'like', 12: 'the', 13: 'keep', 14: 'we', 15: 'do', 16: 'stop', 17: 'please', 18: "don't", 19: 'be', 20: 'come', 21: 'how', 22: 'was', 23: 'my', 24: 'everyone', 25: 'to', 26: 'up', 27: "it's", 28: 'that', 29: 'he', 30: 'she', 31: 'too', 32: 'here', 33: 'lie', 34: 'are', 35: 'no', 36: 'not', 37: 'can', 38: 'go', 39: 'what', 40: 'sorry', 41: 'everybody', 42: 'have', 43: 'very', 44: 'work', 45: 'died', 46: 'left', 47: 'there', 48: 'nobody', 49: 'old', 50: 'cat', 51: "that's", 52: 'try', 53: 'won', 54: 'get', 55: 'in', 56: 'look', 57: 'smiled', 58: 'him', 59: 'still', 60: 'love', 61: "you're", 62: 'blood', 63: 'red', 64: 'singing', 65: 'at', 66: "isn't", 67: 'who', 68: 'got', 69: 'sit', 70: 'home', 71: 'laughed', 72: 'now', 73: 'take', 74: 'turn', 75: 'want', 76: 'yourself', 77: 'fish', 78: 'something', 79: "can't", 80: 'help', 81: 'why', 82: 'us', 83: 'came', 84: 'tell', 85: 'lost', 86: 'hu

In [18]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)             # 입력문을 인코더에 넣어 문장의 상태 벡터를 얻는다
    target_seq = np.zeros((1, 1))                               # 디코더를 초기화한다. 
    target_seq[0, 0] = word_index_target['sos']                 # 디코더의 첫 시작은 <SOS>로 시작하므로 첫 위치에 sos의 인덱스를 기록한다 (원-핫 인코딩 아님)

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:                                   # stop_condition이 True가 될 때까지 루프 반복
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens)

        if (sampled_token_index == 0):                          # 0번 어절은 없으므로, 이때는 sos를 갖는 1로 치환한다
            sampled_token_index = 1

        sampled_word = index_to_tar[sampled_token_index]
        decoded_sentence += sampled_word+' '                    # 마지막에 공백을 하나 붙인다
        
        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_word == 'eos' or len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 길이가 1인 타겟 시퀀스를 업데이트
        target_seq = np.zeros((1, 1))                           # target_input 초기화
        target_seq[0, 0] = sampled_token_index                  # 직전 예측 결과 기록 (원-핫 인코딩 아님)

        # 상태를 업데이트
        states_value = [h, c]

    return decoded_sentence

In [19]:
import numpy as np
for seq_index in [1, 2, 3, 4, 5, 50,60,70,80,90,100]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', data.source[seq_index])
    print('정답 문장:', data.target[seq_index][:len(data.target[seq_index])])
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1].replace('eos', ''))  # 마지막에 붙은 공백과 eos는 빼고 출력


1/1 [==============================] - 0s 14ms/step
-----------------------------------
입력 문장: Hi.
정답 문장: 안녕.
번역기가 번역한 문장: 안녕 
1/1 [==============================] - 0s 15ms/step
-----------------------------------
입력 문장: Run!
정답 문장: 뛰어!
번역기가 번역한 문장: 뛰어 
1/1 [==============================] - 0s 14ms/step
-----------------------------------
입력 문장: Run.
정답 문장: 뛰어.
번역기가 번역한 문장: 뛰어 
1/1 [==============================] - 0s 15ms/step
-----------------------------------
입력 문장: Who?
정답 문장: 누구?
번역기가 번역한 문장: 누가 죽었어 
1/1 [==============================] - 0s 15ms/step
-----------------------------------
입력 문장: Wow!
정답 문장: 우와!
번역기가 번역한 문장: 우와 
1/1 [==============================] - 0s 16ms/step
-----------------------------------
입력 문장: Help me.
정답 문장: 도와줘.
번역기가 번역한 문장: 도와줘 
1/1 [==============================] - 0s 16ms/step
-----------------------------------
입력 문장: Stop it.
정답 문장: 그만해.
번역기가 번역한 문장: 그만해 
1/1 [==============================] - 0s 16ms/step
-----------------------------------
입

In [20]:
# 1개 문장 예측하기
# 전처리
input_seq_1 = tokenizer_source.texts_to_sequences([list('wait!')])
input_seq_1 = pad_sequences(input_seq_1, maxlen=max_src_len, padding='post')

# 예측하기
decoded_sentence = decode_sequence(input_seq_1).replace('eos', '')
print(decoded_sentence)

1/1 [==============================] - 0s 16ms/step
이거 가져  
